In [ ]:
import geopandas as gpd
import pandas as pd
import os

In [ ]:
fp = '/Users/hudsonfinchbatista/Desktop/Python Datasets/Capstone/cb_md/cb_2018_24_bg_500k.shp'

In [ ]:
data = gpd.read_file(fp)

In [ ]:
data.head()

In [ ]:
data1 = data[data['COUNTYFP'] == '031']

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
map = folium.Map(location=[-77.0668, 39.1483],
                     tiles="Cartodb Positron",
                     zoom_start=12,max_bounds=True)

marker_cluster = folium.plugins.MarkerCluster()

In [ ]:
import requests
import json

In [ ]:
census_api_url='https://api.census.gov/data/2019/acs/acs5?key=d62e9dc3485b7ad18ea2f4ebf8a2ebd6a41fd469&get=NAME,B01001_001E,B01002_001E,B19013_001E,B08301_001E,B08301_010E,B15003_001E,B15003_017E,B15003_022E,B23025_003E,B23025_005E&for=block%20group:*&in=state:24&in=county:031&in=tract:*'

responseCensus = requests.get(census_api_url, headers={'Content-Type': 'application/json'})
textCensus=json.loads(responseCensus.content)
tractDF = pd.DataFrame(textCensus)
print('Shape of census tract file: ' + str(tractDF.shape))

tractDF.rename(columns = tractDF.iloc[0], inplace=True)
tractDF = tractDF.iloc[1:]
tractDF.rename(index=str,columns={'B01001_001E':'Pop','B19013_001E':'Income',
                                  'B08301_001E':'Means of Transportation',
                                  'B08301_010E':'Public Transportation',
                                  'B15003_001E':'Educational Attainment','B15003_017E':'High School',
                                  'B15003_022E':'Bachelors',
                                  'B01002_001E':'Age','B23025_003E':'Total Laborforce',
                                  'B23025_005E':'Total Unemployed',
                                  'block group':'Block',
                                  'tract':'Tract'}, inplace=True)

In [ ]:
tractDF['Block'] = tractDF['Block'].apply(str)
tractDF['Tract'] = tractDF['Tract'].apply(str)
tractDF['GEOID'] = ""

state_county = "24031" 

In [ ]:
for i in range(0,len(tractDF)):
    tractDF['GEOID'][i] = state_county + tractDF['Tract'][i] + tractDF['Block'][i]
    
tractDF.head() 

In [ ]:
tractDF.set_index('GEOID', inplace=True)
tractDF.drop(['NAME', 'state', 'county','Tract'],inplace=True, axis=1)
tractDF=tractDF[['Pop','Income','Means of Transportation','Public Transportation','Educational Attainment','High School','Bachelors','Total Laborforce','Total Unemployed','Age']].apply(pd.to_numeric, errors='coerce')
print(tractDF.info())

In [ ]:
data2 = pd.merge(data1,tractDF, how='left', on='GEOID')

In [ ]:
data2[data2['Educational Attainment'] == 0]

In [ ]:
data2 = data2[data2['Educational Attainment'] != 0]

In [ ]:
data2['High School %'] = data2.apply(lambda x: x['High School']/x['Educational Attainment'],axis = 1)
data2['Bachelors %'] = data2.apply(lambda x: x['Bachelors']/x['Educational Attainment'],axis = 1)
data2['Public Transportation %'] = data2.apply(lambda x: x['Public Transportation']/x['Means of Transportation'], axis = 1)
data2['Unemployed %'] = data2.apply(lambda x: x['Total Unemployed']/x['Total Laborforce'], axis = 1)


In [ ]:
data3 = data2[['GEOID','Income','geometry']]

In [ ]:
data3['Income'].value_counts()

In [ ]:
data3 = data3[data3['Income'] != -666666666]

In [ ]:
ax = data3.plot(column='Income',cmap ='YlGnBu')
ax

In [ ]:
os.chdir('/Users/hudsonfinchbatista/Desktop/Python Datasets/Capstone/Crimes by Block')
dataframe5 = pd.read_csv('Crime_Clean.csv')

In [ ]:
for idx, row in dataframe5.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    marker = folium.map.Marker([lat, long])
    marker_cluster.add_child(marker)

In [ ]:
map.add_child(marker_cluster)
map

In [ ]:
data3.to_file('myshpfile.geojson',driver='GeoJSON')

In [ ]:
with open("myshpfile.geojson") as f:
    geo = json.load(f)

In [ ]:
folium.Choropleth(
geo_data=geo,
data=data3,
columns = ['GEOID','Income'],
key_on = 'feature.properties.GEOID',
fill_color = 'OrRd').add_to(map)

In [ ]:
map

In [ ]:
data2.to_file('myshpfile2.geojson',driver='GeoJSON')

with open('myshpfile2.geojson') as g:
    geo2 = json.load(g)

In [ ]:
ax1 = data2.plot(column='Public Transportation %',cmap ='YlGnBu')
ax1

In [ ]:
map2 = folium.Map(location=[-77.0668, 39.1483],
                     tiles="Cartodb Positron",
                     zoom_start=12,max_bounds=True)

marker_cluster2 = folium.plugins.MarkerCluster()

In [ ]:
for idx, row in dataframe5.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    marker = folium.map.Marker([lat, long])
    marker_cluster2.add_child(marker)

In [ ]:
map2.add_child(marker_cluster2)

In [ ]:
folium.Choropleth(
geo_data=geo2,
data=data2,
columns = ['GEOID','Public Transportation %'],
key_on = 'feature.properties.GEOID',
fill_color = 'OrRd').add_to(map2)

In [ ]:
map2

In [ ]:
ax2 = data2.plot(column='Unemployed %',cmap ='YlGnBu')
ax2

In [ ]:
map3 = folium.Map(location=[-77.0668, 39.1483],
                     tiles="Cartodb Positron",
                     zoom_start=12,max_bounds=True)

marker_cluster3 = folium.plugins.MarkerCluster()

In [ ]:
for idx, row in dataframe5.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    marker = folium.map.Marker([lat, long])
    marker_cluster3.add_child(marker)

In [ ]:
map3.add_child(marker_cluster3)

In [ ]:
folium.Choropleth(
geo_data=geo2,
data=data2,
columns = ['GEOID','Unemployed %'],
key_on = 'feature.properties.GEOID',
fill_color = 'OrRd').add_to(map3)

In [ ]:
map3

In [ ]:
ax3 = data2.plot(column='High School %',cmap ='YlGnBu')
ax3

In [ ]:
map4 = folium.Map(location=[-77.0668, 39.1483],
                     tiles="Cartodb Positron",
                     zoom_start=12, max_bounds=True)

marker_cluster4 = folium.plugins.MarkerCluster()

In [ ]:
for idx, row in dataframe5.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    marker = folium.map.Marker([lat, long])
    marker_cluster4.add_child(marker)

In [ ]:
map4.add_child(marker_cluster4)

In [ ]:
folium.Choropleth(
geo_data=geo2,
data=data2,
columns = ['GEOID','High School %'],
key_on = 'feature.properties.GEOID',
fill_color = 'OrRd').add_to(map4)

In [ ]:
map4